In [1]:
from datetime import datetime
import numpy as np
import pandas as pd

In [2]:
# Read State table to get foreign key value
df_ref = pd.read_csv("./cleaned/State.csv")
df_ref.head()

,id,name,country,country_id
0,1,Alabama,United States,224
1,2,Alaska,United States,224
2,3,Arizona,United States,224
3,4,Arkansas,United States,224
4,5,California,United States,224


In [3]:
# Read original CSV
date_parser = lambda x: datetime.strptime(x, '%Y/%m/%d')
df_in = pd.read_csv("./raw/COVID-19_Reported_Patient_Impact_and_Hospital_Capacity_by_State_Timeseries.csv", parse_dates=['date'], date_parser=date_parser)
df_in.head()

,state,date,critical_staffing_shortage_today_yes,critical_staffing_shortage_today_no,critical_staffing_shortage_today_not_reported,critical_staffing_shortage_anticipated_within_week_yes,critical_staffing_shortage_anticipated_within_week_no,critical_staffing_shortage_anticipated_within_week_not_reported,hospital_onset_covid,hospital_onset_covid_coverage,...,previous_day_admission_influenza_confirmed,previous_day_admission_influenza_confirmed_coverage,previous_day_deaths_covid_and_influenza,previous_day_deaths_covid_and_influenza_coverage,previous_day_deaths_influenza,previous_day_deaths_influenza_coverage,total_patients_hospitalized_confirmed_influenza,total_patients_hospitalized_confirmed_influenza_and_covid,total_patients_hospitalized_confirmed_influenza_and_covid_coverage,total_patients_hospitalized_confirmed_influenza_coverage
0,MT,2020-10-03,13,51,3,14,50,3,361.0,65,...,NaN,0,NaN,0,NaN,0,NaN,NaN,0,0
1,IA,2020-10-02,5,116,2,6,115,2,2.0,121,...,NaN,0,NaN,0,NaN,0,NaN,NaN,0,0
2,HI,2020-10-01,3,13,14,3,13,14,4.0,15,...,NaN,0,NaN,0,NaN,0,NaN,NaN,0,0
3,NE,2020-09-27,11,78,4,12,77,4,1.0,88,...,NaN,0,NaN,0,NaN,0,NaN,NaN,0,0
4,MN,2020-09-26,18,93,19,22,90,18,5.0,129,...,NaN,0,NaN,0,NaN,0,NaN,NaN,0,0


In [4]:
df_in["state"].unique()

array(['MT', 'IA', 'HI', 'NE', 'MN', 'RI', 'ID', 'SD', 'MS', 'LA', 'KS',
       'IN', 'NV', 'OR', 'UT', 'ND', 'PR', 'VI', 'CT', 'MA', 'MO', 'DC',
       'NC', 'VA', 'NJ', 'NM', 'VT', 'CO', 'ME', 'DE', 'WV', 'NH', 'WY',
       'AK', 'AS', 'TX', 'AR', 'KY', 'OK', 'NY', 'AL', 'FL', 'PA', 'AZ',
       'CA', 'IL', 'WA', 'WI', 'GA', 'OH', 'MI', 'TN', 'SC', 'MD'],
      dtype=object)

In [5]:
state_abbreviation_mapping = {
    "AK": "Alaska",
    "AL": "Alabama",
    "AR": "Arkansas",
    "AZ": "Arizona",
    "CA": "California",
    "CO": "Colorado",
    "CT": "Connecticut",
    "DE": "Delaware",
    "DC": "District of Columbia",
    "FL": "Florida",
    "GA": "Georgia",
    "HI": "Hawaii",
    "ID": "Idaho",
    "IL": "Illinois",
    "IN": "Indiana",
    "IA": "Iowa",
    "KS": "Kansas",
    "KY": "Kentucky",
    "LA": "Louisiana",
    "ME": "Maine",
    "MD": "Maryland",
    "MA": "Massachusetts",
    "MI": "Michigan",
    "MN": "Minnesota",
    "MS": "Mississippi",
    "MO": "Missouri",
    "MT": "Montana",
    "NE": "Nebraska",
    "NV": "Nevada",
    "NH": "New Hamsphire",
    "NJ": "New Jersey",
    "NM": "New Mexico",
    "NY": "New York",
    "NC": "North Carolina",
    "ND": "North Dakota",
    "OH": "Ohio",
    "OK": "Oklahoma",
    "OR": "Oregon",
    "PA": "Pennsylvania",
    "RI": "Rhode Island",
    "SC": "South Carolina",
    "SD": "South Dakota",
    "TN": "Tennessee",
    "TX": "Texas",
    "UT": "Utah",
    "VT": "Vermont",
    "VA": "Virginia",
    "WA": "Washington",
    "WV": "West Virginia",
    "WI": "Wisconsin",
    "WY": "Wyoming"
}

In [6]:
columns = [
    "record_date",
    "total_beds",
    "total_beds_coverage",
    "covid_occupied_beds",
    "covid_occupied_beds_coverage",
    "icu_beds",
    "icu_beds_coverage",
    "covid_occupied_icu_beds",
    "covid_occupied_icu_beds_coverage",
    "state",
    "state_id"
]
df_out = pd.DataFrame([], columns=columns)
df_out

,record_date,total_beds,total_beds_coverage,covid_occupied_beds,covid_occupied_beds_coverage,icu_beds,icu_beds_coverage,covid_occupied_icu_beds,covid_occupied_icu_beds_coverage,state,state_id


In [7]:
for state_code in df_in["state"].unique():
    state = state_abbreviation_mapping.get(state_code)
    state_ids = df_ref[df_ref['name'] == state]["id"] 
    if(len(state_ids) == 0):
        continue
    state_id = state_ids.iloc[0]
    if(state):
        print("State_id: {}, State: {}".format(state_id, state))
        df_state = df_in[df_in["state"] == state_code].sort_values(by=['date'])
        for i in range(len(df_state)):
            current = df_state.iloc[i]
            record_date = current["date"]
            total_beds = current["inpatient_beds"]
            total_beds_coverage = current["inpatient_beds_coverage"]
            covid_occupied_beds = current["inpatient_beds_used_covid"]
            covid_occupied_beds_coverage = current["inpatient_beds_used_covid_coverage"]
            icu_beds = current["total_staffed_adult_icu_beds"]
            icu_beds_coverage = current["total_staffed_adult_icu_beds_coverage"]
            covid_occupied_icu_beds = current["staffed_icu_adult_patients_confirmed_covid"]
            covid_occupied_icu_beds_coverage = current["staffed_icu_adult_patients_confirmed_covid_coverage"]
            new_row = {
                "record_date": record_date,
                "total_beds": total_beds,
                "total_beds_coverage": total_beds_coverage,
                "covid_occupied_beds": covid_occupied_beds,
                "covid_occupied_beds_coverage": covid_occupied_beds_coverage,
                "icu_beds": icu_beds,
                "icu_beds_coverage": icu_beds_coverage,
                "covid_occupied_icu_beds": covid_occupied_icu_beds,
                "covid_occupied_icu_beds_coverage": covid_occupied_icu_beds_coverage,
                "state": state,
                "state_id": state_id
            }
            df_out = df_out.append(new_row, ignore_index=True)

State_id: 27, State: Montana
State_id: 16, State: Iowa
State_id: 12, State: Hawaii
State_id: 28, State: Nebraska
State_id: 24, State: Minnesota
State_id: 40, State: Rhode Island
State_id: 13, State: Idaho
State_id: 42, State: South Dakota
State_id: 25, State: Mississippi
State_id: 19, State: Louisiana
State_id: 17, State: Kansas
State_id: 15, State: Indiana
State_id: 29, State: Nevada
State_id: 38, State: Oregon
State_id: 45, State: Utah
State_id: 35, State: North Dakota
State_id: 7, State: Connecticut
State_id: 22, State: Massachusetts
State_id: 26, State: Missouri
State_id: 9, State: District of Columbia
State_id: 34, State: North Carolina
State_id: 47, State: Virginia
State_id: 31, State: New Jersey
State_id: 32, State: New Mexico
State_id: 46, State: Vermont
State_id: 6, State: Colorado
State_id: 20, State: Maine
State_id: 8, State: Delaware
State_id: 49, State: West Virginia
State_id: 51, State: Wyoming
State_id: 2, State: Alaska
State_id: 44, State: Texas
State_id: 4, State: Arka

In [15]:
df_out.to_csv("./cleaned/Hospitalization_data.csv", index=False)